# $H_{2}O$ continuum in RRTMG-SW

The absorption coefficients for water continuum in the shortwave are calculated from the following terms:

`selffac`, `selffrac`, `indself`, `selfref`; `forfac`, `forfrac`, `indfor`, `forref`

The prefix or suffix `self` and `for` are for self and foreign continuum, respectively.  

Taking into account of their dimensions and the Fortran file in which they are calculated, these terms can be sorted into 2 categories:

1. `selffac`, `selffrac`, `indself`, `forfac`, `forfrac`, `indfor`.  These have dimensions (layer,).
2. `selfref`, `forref`.  These have dimensions (temperature, g-point).

Below is described how terms in each of the above categories are obtained.

###  `*fac`, `*frac` and `ind*` terms

`selffac`, `selffrac`, `indself`, `forfac`, `forfrac` and `indfor` are calculated in *rrtmg_sw_setcoef.f90*, mainly from the input $H_2O$ concentration,  layer pressure, layer temperature, as well as the column density of dry air.  Here are the relevant lines from *rrtmg_sw_setcoef.f90*:

```
water = wkl(1,lay)/coldry(lay)
```

where 

`wkl` is molecule amount for water $[mol\: cm^{-2}]$  
`coldry` is dry air column density $[mol\: cm^{-2}]$


```
scalefac = pavel(lay) * stpfac / tavel(lay)
```

where

`stpfac` = $296 / 1013$  
`pavel` is layer pressure [mbar]  
`tavel` is layer temperature [K]   


`forfac`, `forfrac`, `selffac` and `selffrac` can now be obtained in terms of `water` and `scalefac`.  The expression used is different between atmosphere layers below and those above $\sim 100$ mbar.

For layers where pressure is greater than ~ 100 mb, these expressions are used:

```
! Set up factors needed to separately include the water vapor                                                           
! foreign-continuum in the calculation of absorption coefficient.                                                       

         forfac(lay) = scalefac / (1.+water)
         factor = (332.0_rb-tavel(lay))/36.0_rb
         indfor(lay) = min(2, max(1, int(factor)))
         forfrac(lay) = factor - real(indfor(lay),kind=rb)
         
! Set up factors needed to separately include the water vapor                                                           
! self-continuum in the calculation of absorption coefficient.                                                          

         selffac(lay) = water * forfac(lay)
         factor = (tavel(lay)-188.0_rb)/7.2_rb
         indself(lay) = min(9, max(1, int(factor)-7))
         selffrac(lay) = factor - real((indself(lay) + 7),kind=rb)
         
```

For layers where pressure is less than ~ 100 mb, these expressions are used:

```
! Set up factors needed to separately include the water vapor                                                           
! foreign-continuum in the calculation of absorption coefficient.                                                       

         forfac(lay) = scalefac / (1.+water)
         factor = (tavel(lay)-188.0_rb)/36.0_rb
         indfor(lay) = 3
         forfrac(lay) = factor - 1.0_rb
         
         selffac(lay) = 0._rb
         selffrac(lay)= 0._rb
         indself(lay) = 0         
```

Notice that there is no self-continuum for layers where pressure are less than 100 mb.

###  `*ref` terms

This section describes how `forref` and `selfref` are obtained.  Essentially, `forref` and `selfref` are pre-computed data of dimensions (temperature, g-point).  

*rrtmg_sw_k_g.f90* contains the data for the absorption coefficients for foreign and self-continuum, named `forrefo` and `selfrefo`, respectively.  These have dimensions (temperature, g-point).  

Along the temperature dimension are temperatures, starting from $245.6$ K, in increments of $7.2$ K. The number of temperature values varies, depending on the spectral band and whether it is self or foreign continuum; there are about 10 values for self and about 4 values for foreign.  

Along the g-point dimension are the g-point indices: 1 for the first g-point, 2 for the second, and so on.  There are 16 g-points in all spectral bands.  Note that these g-points are what is referred to as the original g-points in the comments (hence the `o` at the end of `forrefo` and `selfrefo`).  Since there are 14 spectral bands, there are a total of 224 g-points ---- this original model is also referred to as the 224-g-point model.

`forrefo` and `selfrefo` are defined in separate subroutines for different spectral bands in *rrtmg_sw_k_g.f90*.  For example, `subroutine sw_kgb16` contains `forrefo` and `selfrefo` data for spectral band 16, the first spectral band in the shortwave.

Before being used in the global climate model, this 224-g-point data is first reduced to a simpler 112-g-point set.  This is done by grouping together several g-points in the 224-g-point model into single g-points.  Within the same group, each g-point is given a weight.  This weight, for each g-point in the 224-g-point model, is contained in `rwgt`, which has dimensions (g-point,),  where g-point is the g-point index for all 224 g-points, (1, 2, ..., 224).  

The result of the grouping is as follows.  In the original 224-point model, each spectral band has 16 g-points.  After grouping, each spectral band has fewer g-points but this number may vary from band to band (just that they add up to 112).  The number of g-points in each spectral band in the 112-g-point model is contained in `ngc`.  `ngc` has dimensions (spectral_band,), where spectral_band is the spectral band index (1, 2, ..., 14).  For example, `ngc(3)` is the number of g-points in spectral band 3 in the 112-g-point model.  

The number of g-points in the 224-g-point model that are grouped together to form each g-point in the 112-g-point model is contained in `ngn`.  `ngn` has dimensions (g-point,), where g-point is the g-point index in the 112-g-point model, so has length 112.  For example, `ngn(45)` is the number of g-points in the 224-g-point model that are grouped together to form the 45th g-point in the 112-g-point model.  The g-points that are grouped together are *adjacent* to each other.  So, for example, if `ngn(11) = 5`, then 5 adjacent g-points in the 224-g-point model are grouped together to form the 11th g-point in the 112-g-point model.


The result of this reduction from 224 g-points to 112 g-points are `forref` and `selfref`.  These have dimensions (temperature, g-point).  The values along the temperature dimension are the same as for `forrefo` and `selfrefo`.  The values along the g-point dimension now correpond to the g-points in the 112-gpoint model.  

`forref` and `selfref` are calculated in *rrtmg_sw_init.f90*, with each spectral band in a separate subroutine.  For example, `subroutine cmbgb17` calculates for the 17th spectral band.  In this band, `selfref` is obtained via:

```
      do jt = 1,10
         iprsm = 0
         do igc = 1,ngc(2)
            sumk = 0.
            do ipr = 1, ngn(ngs(1)+igc)
               iprsm = iprsm + 1
               sumk = sumk + selfrefo(jt,iprsm)*rwgt(iprsm+16)
            enddo
            selfref(jt,igc) = sumk
         enddo
      enddo
```
Some comments.  `jt` loops over the temperature dimension.  `igc` loops over the `ngc(2)` g-points in the 17th (or 2nd in the shortwave) spectral band in the 112-g-point model.  `ipr` loops over the `ngn(ngs(1)+igc)` g-points in the 224-g-point model that are grouped together to form g-point `igc` in spectral band 17 in the 112-g-point model. `ngs(1)` is the total number of g-points in spectral band 1 and the bands before it, in the 112-g-point model.  `iprsm` is the index of the g-point in spectral band 17 in the 224-g-point model, starting from 1 for the first g-point in spectral band 17.  The index for `rwgt` is `iprsm + 16` because its index starts from the first g-point in the *first* spectral band in the 224-g-point model rather than from the first g-point in the current spectral band.

`forref` is calculated the same way, just replace `selfrefo` by `forrefo` and `selfref` by `forref`.  The weight `rwgt` used is the same.




### Absorption coefficient

Once the following terms:

`selffac`, `selffrac`, `indself`, `selfref`  
`forfac`, `forfrac`, `indfor`, `forref`

are all obtained, they are combined to form the absorption coefficients.  This is done in *rrtmg_sw_taumol.f90* via:

```
                 (selffac(lay) * (selfref(inds,ig) + &
                 selffrac(lay) * &
                 (selfref(inds+1,ig) - selfref(inds,ig))) + &
                 
                 forfac(lay) * (forref(indf,ig) + &
                 forfrac(lay) * &
                 (forref(indf+1,ig) - forref(indf,ig))))
```

where 

```
inds = indself(lay)
indf = indfor(lay)
```
.

The absorption coefficient is a function of layer and g-point.  The above expression calculates the aborption coefficient for layer `lay` and g-point `ig`.  `indself` maps the atmosphere layers to the temperatures of `selfref`, while `indfor` maps the atmosphere layers to the temperatures of `forref`.  

The expressions for self and foreign continuum have the same form.  The spectral dependence comes from the `*-ref` terms' g-points dimension, while the layer dependence comes from both the layer dimension and the temperature dimension of all terms:

```
      fac(lay) * (ref(lay, ig) 
                  + frac(lay) * (ref(lay+1, ig) - ref(lay, ig))
                 )

```
This same form of expression is used wherever water continuum is to be included, in all spectral bands and in all low and high atmosphere regions.
